# ADO for CDN

We used the [ADOpy](https://github.com/adopy/adopy) to inspire our development of ADO for CDN. In this notebook we go over the mathematical foundation underlying ADO.

## Introduction to grid-based ADO algorithm

ADO works by first pre-computing the log-likelihood, entropy, and prior at step 0. Then the optimization takes place iteratively over three steps by updating the mutual information after a choice is made. This theoretical framework will be written in general for any given task design. In practice, a lot of computations can be written generally to work for any task design. The initial computation of the log-likelihood is based on each task and $SV$ defined by the model.


### Step 0: Pre-computation

In each task, or experiment, the participant makes a decision or a choice, $y$. This data is fit to a model defined by parameters, $\theta$. Finally, the design or choices presented to the participant is defined by the choice set space, $d$. In the code, these three values are defined first to subsequently compute the following distributions, etc. 



<span style="color:yellow">1. Precompute the log-likelihood $log(p(y|\theta,d))$ for all values in $y$,$\theta$, and $d$:</span>.


Given each possible combination ($y_i$,$\theta_j$,$d_k$), compute $SV_{reward}$ and $SV_{null}$ defined by the model. Use the difference between the two $\Delta SV = SV_{reward} - SV_{null}$ to get a probability, $p_l$, given a $\gamma_j$ (taken from $\theta_j$). Finally compare probability $p_l$ to choice $y_i$ and compute the $log(p(y_i|\theta_j,d_k))$. In Python you can compute `log_lik` using `choice` as $y_i$ `p_choose_reward` as $p_l$ by hand or using `bernoulli.logpmf` from `scipy.stats`:

`log_lik = (choice==1)*np.log(p_choose_reward) + (choice==0)*np.log(1-p_choose_reward)`

`log_lik = bernoulli.logpmf(choice, p_choose_reward)`



2. Precompute the entropy $H(Y(d)|\theta) = -\sum_{y} p(y|\theta,d) log(p(y|\theta,d))$ for all values of $d$ and $\theta$:

We use the log-likelihood (`log_lik`) to compute entropy as the second term in the summation is the `log_lik`, while the first term is the likelihood, so we exponentiate $p(y|\theta,d) = e^{log(p(y|\theta,d))}$. In the code this is computed using Numpy's function [`np.einsum()`](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html)

`ent = -1 * np.einsum('dpy,dpy->dp', np.exp(log_lik), log_lik)`

Briefly, the function `np.einsum()` uses Einstein summation convention to sum over $y$. The first term is a string indicating what axis you intend to sum over. In this case `dpy` refers to design, parameters, output/response. Then the notation used `dpy,dpy-> dp` indicates summing along $y$, axis=$2$ from $(0,1,2)$.




3. Initialize prior $p_t(\theta)$ for each discretized values of $\theta$:

If you have no priors then you can set this to a uniform distribution, normalized to $1.0$ as follows:

`log_prior = np.log(np.ones(n_p, dtype=np.float32) / n_p)`

where `n_p` is the number of parameters ($\theta_j$). If you have a `prior` that you want to initialize, you can define an array with zeros, then set the parameter index you want to initialize to 1.0. Then convert this prior to a `log_prior` by trapping the prior into a `log_prior`. This is part of the code paraphrased here for illustration:


In [1]:
import numpy as np
# here we think parameter index 12 is the one we want to initialize
pmid_idx = 12
n_p = 30 # this is number of parameters computed elsewhere
prior = np.zeros(n_p, dtype=np.float32)
prior[pmid_idx] = 1
noise_ratio = 1e-7
log_prior = np.log((1 - 2 * noise_ratio) * prior + noise_ratio)
print('log_prior is :\n {}'.format(log_prior))

log_prior is :
 [-1.6118095e+01 -1.6118095e+01 -1.6118095e+01 -1.6118095e+01
 -1.6118095e+01 -1.6118095e+01 -1.6118095e+01 -1.6118095e+01
 -1.6118095e+01 -1.6118095e+01 -1.6118095e+01 -1.6118095e+01
 -5.9604645e-08 -1.6118095e+01 -1.6118095e+01 -1.6118095e+01
 -1.6118095e+01 -1.6118095e+01 -1.6118095e+01 -1.6118095e+01
 -1.6118095e+01 -1.6118095e+01 -1.6118095e+01 -1.6118095e+01
 -1.6118095e+01 -1.6118095e+01 -1.6118095e+01 -1.6118095e+01
 -1.6118095e+01 -1.6118095e+01]


### Step 1. Design Optimization

After pre-computing the above distributions, we begin the optimization phase. In this first step, we compute several different distributions to allow us to compute the mutal information which is maximized to find the new design.

1. Compute the marginal likelihood $p(y|d) = \sum_{\theta} p(y|\theta,d)p_t(\theta)$ for all values of $y$ and $d$.





2. Compute cond ent



3. Computer marg entropy



4. ID optimal design to max MI

### Step 2. Experimentation

Simulate or ask for input

### Step 3. Bayesian Update

1. Update posterior

2. Set new